In [191]:
# ce code permet d'extraire l'information si le produit est végétarien ou non: 
# on va ensuite généraliser le calcul pour touts les produits:
from bs4 import BeautifulSoup
import requests

url = "https://fr.openfoodfacts.org/produit/8000500310427/nutella-biscuits"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

h4_tags = soup.find_all('h4')

found_vegetarian_info = False

vegetarian_info=[]
for tag in h4_tags:
    if "Végétarien" in tag.text:
        vegetarien_text = tag.text.strip()
        print(vegetarien_text)
        found_vegetarian_info = True
        vegetarian_info.append('Végétarien')
        break
    elif "Non végétalien" in tag.text:
        non_vegetalien_text = tag.text.strip()
        print(non_vegetalien_text)
        found_vegetarian_info = True
        vegetarian_info.append('Non végétarien')
        break

if not found_vegetarian_info:
    print("Information végétarienne non trouvée")
    vegetarian_info.append('NON')

vegetarian_info= vegetarian_info[0]

Non végétalien


## Scrapper 1000 produits (100 produits*10 pages):

In [227]:
def scrap_openfoodfacts(nb_pages=10):
     """ Il s'agit de la fonction principale du module.
    Cette dernière crée dans votre espace de travail un DataFrame Pandas contenant les informations scrapées sur le site OpenFoodFacts.
    L'argument "nb_pages" permet de régler le nombre de page à scraper.
    Veuillez ne pas trop l'augmenter afin que l'opération prenne un temps raisonnable.
    Il faut compter environ 30 secondes pour scraper une page (25 minutes pour les 50 pages par défaut).
    6 variables sont scrapées pour chaque nouvelle donnée.
    """
    
    # Mesure du temps
    start_time = time.time()
    records = []
    # Initialisation de la liste records récoltant nos données
    error = np.NaN
   # On récupère l'url de chaque produit sur le nombre de pages souhaitées
    for i in range(1, nb_pages + 1):
        r = requests.get('https://fr.openfoodfacts.org/' + str(i))
        soup = BeautifulSoup(r.text, 'html.parser')

        products = soup.find_all('ul', {'class': "products"})
        products = products[0].find_all('a')

        liste_url = ['https://fr.openfoodfacts.org/' + elt['href'] for elt in products]
        
 # Pour chaque produit on place dans des variables les données que l'on souhaite scraper
        for url in liste_url:
            s = requests.get(url)
            soup = BeautifulSoup(s.text, 'html.parser')
            # Si la donnée peut être récupérée, on la place dans notre variable, sinon on la replace par une erreur 
            try:            
                name = soup.title.text[:-2]
            except:
                name = error

            try:
                code_barre = soup.find('span', attrs = {'style' : "speak-as:digits;"}).text
            except:
                code_barre = error

            h4_tags = soup.find_all('h4')

            found_vegetarian_info = False

            vegetarian_info=[]
            h4_tags = soup.find_all('h4')

            found_vegetarian_info = False

            vegetarian_info=[]
            for tag in h4_tags:
                if "Végétarien" in tag.text:
                    vegetarien_text = tag.text.strip()
                    found_vegetarian_info = True
                    vegetarian_info.append('Végétarien')
                    break
                elif "Non végétalien" in tag.text:
                    non_vegetalien_text = tag.text.strip()
                    found_vegetarian_info = True
                    vegetarian_info.append('Non végétarien')
                    break

            if not found_vegetarian_info:
                vegetarian_info.append('NON')

            vegetarian_info= vegetarian_info[0]

            try:
                Pays_de_ventes=soup.find('p', {'id': 'field_countries'}).find('span', {'class': 'field_value'}).text.strip()
            except:
                origine = error

            try:
                caractéristiques = soup.find(itemprop="description").text
            except:
                caractéristiques = error

            try:             
                quantity = soup.find('p', {'id': 'field_quantity'}).find('span', {'class': 'field_value'}).text.strip()
            except: 
                quantity = 'NON'
    
            records.append((name, code_barre, quantity, vegetarian_info, Pays_de_ventes, caractéristiques))

            i += 1

        sleep(1)
        
        # On construit le DataFrame, puis on l'exporte dans l'espace de travail

    df = pd.DataFrame(records, columns=['Produit', 'CodeBarre', 'Quantité', 'vegetarian_info', 'Pays_de_ventes', 'Caractéristiques'])

    return df


In [231]:
df = scrap_openfoodfacts(nb_pages=10)

## Exporter le dataset dans un csv:

In [232]:
df.to_csv('./OpenFoodFacts.csv', index=False, encoding='utf-8')  # Exportation de dataframe dans un csv

In [233]:
df

,Produit,CodeBarre,Quantité,vegetarian_info,Pays_de_ventes,Caractéristiques
0,"Eaux de sources - Cristaline - 1,5",3274080005003,"1,5 L",Végétarien,"Belgique, Côte d'Ivoire, France, Allemagne, Gu...",Pisciazza
1,Pâte à tartiner Nutella noisettes et cacao - F...,3017620422003,400 g,Non végétarien,"Belgique, France, Allemagne, Italie, Luxembour...",Pâte à tartiner aux noisettes
2,Prince Chocolat biscuits - Lu - 300,7622210449283,300 g,Non végétarien,"Algérie, Belgique, France, Polynésie française...",BISCUITS FOURRÉS (35%) PARFUM CHOCOLAT
3,Coca-Cola goût original - 330,5449000214911,330 mL,NON,"Belgique, France, Allemagne, Hongrie, Italie, ...",Boisson rafraîchissante aux extraits végétaux
4,Nutella - Ferrero - 1,3017620425035,1 kg,Non végétarien,"Algérie, Belgique, France, Allemagne, Guadelou...",Pâte à tartiner aux noisettes
...,...,...,...,...,...,...
995,"Paulaner Spezi - 0,",4066600603405,"0,5l",Végétarien,"France, Allemagne, Suisse",NaN
996,Coca-Cola Zéro sucres - 50,5000112558265,50 cl,NON,"Belgique, France, Allemagne, Suisse","Boisson rafraîchissante aux extraits végétaux,..."
997,NESTLE CHOCAPIC Céréales 1kg - 1000,7613032779566,1000 g,NON,"France, Suisse",Céréales aromatisées prêtes à consommer enrich...
998,Corn Flakes - Kellogg's - 375,8010265640202,375 g,NON,"France, Grèce, Italie, Espagne",NaN
